# Comparison of spectrum between eNATL60 outputs and Altika Satellite for Region 1 and ASO months

Requisites :
 - notebook process-spectrum-comparison-eNATL60-region1-Altkia-ASO has successfully run and produced all the necessary result_*.nc in results_Altika-eNATL60-Region1-ASO
 - gonzag_cloud
 - climporn
 


In [1]:
import sys,os
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
GONZAG_DIR = '/home/jovyan/gonzag_cloud'
sys.path.append(GONZAG_DIR)
import gonzag as gz


In [3]:
import xarray as xr
import sys
import glob
import numpy as nmp
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker

%matplotlib inline


In [4]:
CLIMPORN_DIR = '/home/jovyan/climporn/python' ; # get it there: https://github.com/brodeau/climporn
sys.path.append(CLIMPORN_DIR)
import climporn as cp


### Params

In [5]:
region = '1'
season = 'fma'


For the model 1

In [6]:
model1 = 'eNATL60'
name_mod1 = model1+'-Region'+region+'-'+season
name_ssh_mod1 = 'sossheig_bl'


For the model 2

In [7]:
model2 = 'GIGATL'
name_mod2 = model2+'-Region'+region+'-'+season
name_ssh_mod2 = 'zeta_bl'


In [8]:
name_mods = model1+'-'+model2+'-Region'+region+'-'+season


For altimetry data

In [9]:
name_sat= 'Altika'
name_ssh_sat='sla_unfiltered'


### Data

In [10]:
fresults1=sorted(glob.glob('../results/results_'+name_sat+'-'+name_mod1+'/result_??.nc'))
dsn1=xr.open_mfdataset(fresults1,concat_dim='time',combine='nested')

In [11]:
fresults2=sorted(glob.glob('../results/results_'+name_sat+'-'+name_mod2+'/result_??.nc'))
dsn2=xr.open_mfdataset(fresults2,concat_dim='time',combine='nested')

In [12]:
#clean up some remaining Nans

ds1=dsn1.where(nmp.isnan(dsn1[name_ssh_sat])==0, drop=True)
ds2=dsn2.where(nmp.isnan(dsn2[name_ssh_sat])==0, drop=True)

### Spectrum

In [13]:
%%time

ISeg_beg1, ISeg_end1 = gz.FindUnbrokenSegments( ds1.time, ds1.distance, ds1[name_ssh_mod1],\
                                             rcut_time=1.2e+09, rcut_dist=7.8 )
ISeg_beg2, ISeg_end2 = gz.FindUnbrokenSegments( ds2.time, ds2.distance, ds2[name_ssh_mod2],\
                                             rcut_time=1.2e+09, rcut_dist=7.8 )
NbSeg1, Nsl1, IDEDSeg1 = gz.SegmentSelection(ISeg_beg1, ISeg_end1, np_valid_seg=70)
NbSeg2, Nsl2, IDEDSeg2 = gz.SegmentSelection(ISeg_beg2, ISeg_end2, np_valid_seg=70)
XPs1, XPm1, rdist_sample1 = gz.Process4FFT( IDEDSeg1, ds1.distance, ds1[name_ssh_mod1], ds1[name_ssh_sat] )
XPs2, XPm2, rdist_sample2 = gz.Process4FFT( IDEDSeg2, ds2.distance, ds2[name_ssh_mod2], ds2[name_ssh_sat] )
Kwn1, PwSpc_s1, PwSpc_m1 = gz.ApplyFFT( IDEDSeg1, XPs1, XPm1, rdist_sample1 )
Kwn2, PwSpc_s2, PwSpc_m2 = gz.ApplyFFT( IDEDSeg2, XPs2, XPm2, rdist_sample2 )

# Building our spectrum as the mean of the NbSeg spectra:
vps_mod1 = nmp.mean(PwSpc_m1[:,:],axis=0)
vps_mod2 = nmp.mean(PwSpc_m2[:,:],axis=0)
vps_sat = nmp.mean(PwSpc_s1[:,:],axis=0)



 *** [SegmentSelection()]: Will use 23 segments with a fixed length of 100 points!
     ==> 23 selected segments out of the 240 available (requested minimum length is 70 points)

 *** [SegmentSelection()]: Will use 105 segments with a fixed length of 80 points!
     ==> 105 selected segments out of the 534 available (requested minimum length is 70 points)

 *** [Process4FFT()]: applying linear detrending...
 *** [Process4FFT()]: centering about 0...
 *** [Process4FFT()]: applying "Tukey" tappering...

 *** [Process4FFT()]: applying linear detrending...
 *** [Process4FFT()]: centering about 0...
 *** [Process4FFT()]: applying "Tukey" tappering...

 *** [ApplyFFT()]: Applying FFT with a dx_sample of  6.603  km

 *** [ApplyFFT()]: Applying FFT with a dx_sample of  6.603  km

CPU times: user 14min 21s, sys: 58.7 s, total: 15min 19s
Wall time: 14min 10s


In [15]:
# Blabla for the plot:
clr_sat = '#AD0000'
clr_mod1 = '#008ab8'
clr_mod2 = '#329E46'

cinfrm = str(NbSeg1)+' segments for '+model1+'\n'+str(NbSeg2)+' segments for '+model2+'\n'+str(Nsl1)+' points/segment for '+model1+'\n'+str(Nsl1)+' points/segment for '+model1+r'$\Delta$d sat.: '+str(round(rdist_sample1,1))+' km'

ii = cp.plot("pow_spectrum_ssh")(Kwn1, vps_mod1, clab1=name_mod1+' ("'+name_ssh_mod1+'")', clr1=clr_mod1, lw1=5, \
                                 cinfo=cinfrm, logo_on=False, \
                                 L_min=13., L_max=500., P_min_y=-6, P_max_y=1, \
                                 l_show_k4=False, l_show_k5=True, l_show_k11o3=False, l_show_k2=True, \
                                 vk2=Kwn1, vps2=vps_sat, clab2=name_sat+' ("'+name_ssh_sat+'")', clr2=clr_sat, lw2=4, \
                                 vk3=Kwn2, vps3=vps_mod2, clab3=name_mod2+' ("'+name_ssh_mod2+'")', clr3=clr_mod2, lw3=4)

plt.savefig('../plots/spectrum_SSH_'+name_sat+'-'+name_mods+'.png')


findfont: Font family ['Trebuchet MS'] not found. Falling back to DejaVu Sans.
